<a href="https://colab.research.google.com/github/Sopralapanca/CommonLitChallenge/blob/main/Correction%2BPOS%2BNER%2BPrepareDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAVORO PER SARDE:
1. Aggiungere altri todo
2. Imparare ad essere umile
3. Tifare il Campobasso

TODO: CONTROLLARE I PARAGRAFI

1. Controllare se non ci sono whitespace dopo gli .\n Maiuscola
2. Score value spacy per identificare la similarità, potremmo usarla per paragrafi diversi.

TODO: CONTROLLARE I NOMI PROPRI
1. Si potrebbe effettuare il lower() prima di aggiungere le parole allo spellchecker sperando che ci siano i nomi propri necessari alla stesura del riassunto all'interno del prompt
2. Controllare i nomi propri facendo ner e non tenerli in considerazione per eventuali errori, ma confrontarli con eventuali nomi propri all'interno dei prompt (selezionare i soggetti che iniziano per le stesse lettere e poi calcolarcisi l'edit distance ==> wording, entro un certo range viene considerato match e si salva una feature per il numero di match con i prompt ==> content)



## Data importing and setup

In [ ]:
!pip install pyspellchecker sympy spacy autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=6d62c50dcdc7522638a48f626af1503de30c7655752e37d0eb2b65370937b6c9
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [ ]:
!python -m spacy download en_core_web_sm

2023-09-02 09:18:12.466657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 09:18:13.937039: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!git clone "https://github.com/Sopralapanca/CommonLitChallenge.git"

Cloning into 'CommonLitChallenge'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 235 (delta 46), reused 58 (delta 44), pack-reused 171
Receiving objects: 100% (235/235), 16.33 MiB | 12.66 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [ ]:
import pandas as pd
summaries_train_path = "/content/CommonLitChallenge/data/summaries_train.csv"
prompt_train_path = "/content/CommonLitChallenge/data/prompts_train.csv"

train_data = pd.read_csv(summaries_train_path, sep=',', index_col=0)
prompt_data = pd.read_csv(prompt_train_path, sep=',', index_col=0)

## Spell Correction

### Creating the list of words

In [ ]:
import itertools
word_list = []
for prompt, query, title in zip(prompt_data.prompt_text.tolist(), prompt_data.prompt_question.tolist(), prompt_data.prompt_title.tolist()):
  word_list.append(prompt.replace('\n', ' ').split())
  word_list.append(query.replace('\n', ' ').split())
  word_list.append(title.replace('\n', ' ').split())
token_list = list(itertools.chain(*word_list))

### Correction of the summaries

In [ ]:
from autocorrect import Speller

class SpellCorrector():
  def __init__(self) -> None:
    self.speller = Speller(lang='en')
  def initialize_tokens(self, tokens):
    self.speller.nlp_data.update({token:1000 for token in tokens})
  def run(self, dataframe):
    dataframe["fixed_summary_text"] = dataframe["text"].apply(
            lambda x: self.speller(x)
        )
    return dataframe["fixed_summary_text"]

In [ ]:
spell_enchanter = SpellCorrector()
spell_enchanter.initialize_tokens(token_list)
train_data["corrected_text"] = spell_enchanter.run(train_data)

## Text processing

In [ ]:
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocessText(text, removal=True):
    # replace newline with space
    text = text.replace("\n", " ")
    text = text.replace('\r', ' ')

    # Normalize spaces around punctuation marks
    text = re.sub(r"[^A-Za-z0-9']", r' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Replace curly apostrophe with straight single quote
    text = text.replace('’', "'")

    # Normalize spaces around punctuation marks
    text = text.strip()

    # lower case
    text = text.lower()

    if removal:
      # split text
      words = text.split()

      # stop word removal
      words = [w for w in words if not w in stop_words]

      # stemming
      # words = [stemmer.stem(w) for w in words]

      # lemmatization
      words = [lemmatizer.lemmatize(w) for w in words]

      text = ' '.join(words)

    return text

# Preprocess the text
prompt_data["prompt_text_preprocessed"] =   prompt_data["prompt_text"].apply(lambda x: preprocessText(x))
train_data["corrected_text_preprocessed"] = train_data["corrected_text"].apply(lambda x: preprocessText(x))

train_data["text_pre_withstop"] =           train_data["text"].apply(lambda x: preprocessText(x, False))
train_data["corrected_text_pre_withstop"] = train_data["corrected_text"].apply(lambda x: preprocessText(x, False))
# Count the length of text
prompt_data["prompt_text_length"] = prompt_data["prompt_text"].apply(len)
train_data["text_length"] =         train_data["corrected_text"].apply(len)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


### N-grams finding

In [ ]:
from nltk.util import ngrams
from collections import Counter
import nltk
nltk.download('punkt')  # Download the required resources for tokenization

def count_ngrams(text, n):
  words = nltk.word_tokenize(text)
  ngram_counts = Counter(ngrams(words, n))
  return ngram_counts

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
del_columns = []

for n in range(2,5):
  col = f"{n}grams-prompttext-count"
  prompt_data[col] = prompt_data.apply(lambda row: count_ngrams(row["prompt_text"], n), axis=1)
  del_columns.append(col)

  col = f"{n}grams-correctedtext-count"
  train_data[col] = train_data.apply(lambda row: count_ngrams(row["corrected_text"], n), axis=1)
  del_columns.append(col)

  col = f"{n}grams-text-count"
  train_data[col] = train_data.apply(lambda row: count_ngrams(row["text"], n), axis=1)
  del_columns.append(col)

### POS counter

In [ ]:
# @title Glossary def.
GLOSSARY = {
    # POS tags
    # Universal POS Tags
    # http://universaldependencies.org/u/pos/
    "ADJ": "adjective",
    "ADP": "adposition",
    "ADV": "adverb",
    "AUX": "auxiliary",
    "CONJ": "conjunction",
    "CCONJ": "coordinating conjunction",
    "DET": "determiner",
    "INTJ": "interjection",
    "NOUN": "noun",
    "NUM": "numeral",
    "PART": "particle",
    "PRON": "pronoun",
    "PROPN": "proper noun",
    "PUNCT": "punctuation",
    "SCONJ": "subordinating conjunction",
    "SYM": "symbol",
    "VERB": "verb",
    "X": "other",
    "EOL": "end of line",
    "SPACE": "space",
    # POS tags (English)
    # OntoNotes 5 / Penn Treebank
    # https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    ".": "punctuation mark, sentence closer",
    ",": "punctuation mark, comma",
    "-LRB-": "left round bracket",
    "-RRB-": "right round bracket",
    "``": "opening quotation mark",
    '""': "closing quotation mark",
    "''": "closing quotation mark",
    ":": "punctuation mark, colon or ellipsis",
    "$": "symbol, currency",
    "#": "symbol, number sign",
    "AFX": "affix",
    "CC": "conjunction, coordinating",
    "CD": "cardinal number",
    "DT": "determiner",
    "EX": "existential there",
    "FW": "foreign word",
    "HYPH": "punctuation mark, hyphen",
    "IN": "conjunction, subordinating or preposition",
    "JJ": "adjective (English), other noun-modifier (Chinese)",
    "JJR": "adjective, comparative",
    "JJS": "adjective, superlative",
    "LS": "list item marker",
    "MD": "verb, modal auxiliary",
    "NIL": "missing tag",
    "NN": "noun, singular or mass",
    "NNP": "noun, proper singular",
    "NNPS": "noun, proper plural",
    "NNS": "noun, plural",
    "PDT": "predeterminer",
    "POS": "possessive ending",
    "PRP": "pronoun, personal",
    "PRP$": "pronoun, possessive",
    "RB": "adverb",
    "RBR": "adverb, comparative",
    "RBS": "adverb, superlative",
    "RP": "adverb, particle",
    "TO": 'infinitival "to"',
    "UH": "interjection",
    "VB": "verb, base form",
    "VBD": "verb, past tense",
    "VBG": "verb, gerund or present participle",
    "VBN": "verb, past participle",
    "VBP": "verb, non-3rd person singular present",
    "VBZ": "verb, 3rd person singular present",
    "WDT": "wh-determiner",
    "WP": "wh-pronoun, personal",
    "WP$": "wh-pronoun, possessive",
    "WRB": "wh-adverb",
    "SP": "space (English), sentence-final particle (Chinese)",
    "ADD": "email",
    "NFP": "superfluous punctuation",
    "GW": "additional word in multi-word expression",
    "XX": "unknown",
    "BES": 'auxiliary "be"',
    "HVS": 'forms of "have"',
    "_SP": "whitespace",
    # POS Tags (German)
    # TIGER Treebank
    # http://www.ims.uni-stuttgart.de/forschung/ressourcen/korpora/TIGERCorpus/annotation/tiger_introduction.pdf
    "$(": "other sentence-internal punctuation mark",
    "$,": "comma",
    "$.": "sentence-final punctuation mark",
    "ADJA": "adjective, attributive",
    "ADJD": "adjective, adverbial or predicative",
    "APPO": "postposition",
    "APPR": "preposition; circumposition left",
    "APPRART": "preposition with article",
    "APZR": "circumposition right",
    "ART": "definite or indefinite article",
    "CARD": "cardinal number",
    "FM": "foreign language material",
    "ITJ": "interjection",
    "KOKOM": "comparative conjunction",
    "KON": "coordinate conjunction",
    "KOUI": 'subordinate conjunction with "zu" and infinitive',
    "KOUS": "subordinate conjunction with sentence",
    "NE": "proper noun",
    "NNE": "proper noun",
    "PAV": "pronominal adverb",
    "PROAV": "pronominal adverb",
    "PDAT": "attributive demonstrative pronoun",
    "PDS": "substituting demonstrative pronoun",
    "PIAT": "attributive indefinite pronoun without determiner",
    "PIDAT": "attributive indefinite pronoun with determiner",
    "PIS": "substituting indefinite pronoun",
    "PPER": "non-reflexive personal pronoun",
    "PPOSAT": "attributive possessive pronoun",
    "PPOSS": "substituting possessive pronoun",
    "PRELAT": "attributive relative pronoun",
    "PRELS": "substituting relative pronoun",
    "PRF": "reflexive personal pronoun",
    "PTKA": "particle with adjective or adverb",
    "PTKANT": "answer particle",
    "PTKNEG": "negative particle",
    "PTKVZ": "separable verbal particle",
    "PTKZU": '"zu" before infinitive',
    "PWAT": "attributive interrogative pronoun",
    "PWAV": "adverbial interrogative or relative pronoun",
    "PWS": "substituting interrogative pronoun",
    "TRUNC": "word remnant",
    "VAFIN": "finite verb, auxiliary",
    "VAIMP": "imperative, auxiliary",
    "VAINF": "infinitive, auxiliary",
    "VAPP": "perfect participle, auxiliary",
    "VMFIN": "finite verb, modal",
    "VMINF": "infinitive, modal",
    "VMPP": "perfect participle, modal",
    "VVFIN": "finite verb, full",
    "VVIMP": "imperative, full",
    "VVINF": "infinitive, full",
    "VVIZU": 'infinitive with "zu", full',
    "VVPP": "perfect participle, full",
    "XY": "non-word containing non-letter",
    # POS Tags (Chinese)
    # OntoNotes / Chinese Penn Treebank
    # https://repository.upenn.edu/cgi/viewcontent.cgi?article=1039&context=ircs_reports
    "AD": "adverb",
    "AS": "aspect marker",
    "BA": "把 in ba-construction",
    # "CD": "cardinal number",
    "CS": "subordinating conjunction",
    "DEC": "的 in a relative clause",
    "DEG": "associative 的",
    "DER": "得 in V-de const. and V-de-R",
    "DEV": "地 before VP",
    "ETC": "for words 等, 等等",
    # "FW": "foreign words"
    "IJ": "interjection",
    # "JJ": "other noun-modifier",
    "LB": "被 in long bei-const",
    "LC": "localizer",
    "M": "measure word",
    "MSP": "other particle",
    # "NN": "common noun",
    "NR": "proper noun",
    "NT": "temporal noun",
    "OD": "ordinal number",
    "ON": "onomatopoeia",
    "P": "preposition excluding 把 and 被",
    "PN": "pronoun",
    "PU": "punctuation",
    "SB": "被 in short bei-const",
    # "SP": "sentence-final particle",
    "VA": "predicative adjective",
    "VC": "是 (copula)",
    "VE": "有 as the main verb",
    "VV": "other verb",
    # Noun chunks
    "NP": "noun phrase",
    "PP": "prepositional phrase",
    "VP": "verb phrase",
    "ADVP": "adverb phrase",
    "ADJP": "adjective phrase",
    "SBAR": "subordinating conjunction",
    "PRT": "particle",
    "PNP": "prepositional noun phrase",
    # Dependency Labels (English)
    # ClearNLP / Universal Dependencies
    # https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md
    "acl": "clausal modifier of noun (adjectival clause)",
    "acomp": "adjectival complement",
    "advcl": "adverbial clause modifier",
    "advmod": "adverbial modifier",
    "agent": "agent",
    "amod": "adjectival modifier",
    "appos": "appositional modifier",
    "attr": "attribute",
    "aux": "auxiliary",
    "auxpass": "auxiliary (passive)",
    "case": "case marking",
    "cc": "coordinating conjunction",
    "ccomp": "clausal complement",
    "clf": "classifier",
    "complm": "complementizer",
    "compound": "compound",
    "conj": "conjunct",
    "cop": "copula",
    "csubj": "clausal subject",
    "csubjpass": "clausal subject (passive)",
    "dative": "dative",
    "dep": "unclassified dependent",
    "det": "determiner",
    "discourse": "discourse element",
    "dislocated": "dislocated elements",
    "dobj": "direct object",
    "expl": "expletive",
    "fixed": "fixed multiword expression",
    "flat": "flat multiword expression",
    "goeswith": "goes with",
    "hmod": "modifier in hyphenation",
    "hyph": "hyphen",
    "infmod": "infinitival modifier",
    "intj": "interjection",
    "iobj": "indirect object",
    "list": "list",
    "mark": "marker",
    "meta": "meta modifier",
    "neg": "negation modifier",
    "nmod": "modifier of nominal",
    "nn": "noun compound modifier",
    "npadvmod": "noun phrase as adverbial modifier",
    "nsubj": "nominal subject",
    "nsubjpass": "nominal subject (passive)",
    "nounmod": "modifier of nominal",
    "npmod": "noun phrase as adverbial modifier",
    "num": "number modifier",
    "number": "number compound modifier",
    "nummod": "numeric modifier",
    "oprd": "object predicate",
    "obj": "object",
    "obl": "oblique nominal",
    "orphan": "orphan",
    "parataxis": "parataxis",
    "partmod": "participal modifier",
    "pcomp": "complement of preposition",
    "pobj": "object of preposition",
    "poss": "possession modifier",
    "possessive": "possessive modifier",
    "preconj": "pre-correlative conjunction",
    "prep": "prepositional modifier",
    "prt": "particle",
    "punct": "punctuation",
    "quantmod": "modifier of quantifier",
    "rcmod": "relative clause modifier",
    "relcl": "relative clause modifier",
    "reparandum": "overridden disfluency",
    "root": "root",
    "ROOT": "root",
    "vocative": "vocative",
    "xcomp": "open clausal complement",
    # Dependency labels (German)
    # TIGER Treebank
    # http://www.ims.uni-stuttgart.de/forschung/ressourcen/korpora/TIGERCorpus/annotation/tiger_introduction.pdf
    # currently missing: 'cc' (comparative complement) because of conflict
    # with English labels
    "ac": "adpositional case marker",
    "adc": "adjective component",
    "ag": "genitive attribute",
    "ams": "measure argument of adjective",
    "app": "apposition",
    "avc": "adverbial phrase component",
    "cd": "coordinating conjunction",
    "cj": "conjunct",
    "cm": "comparative conjunction",
    "cp": "complementizer",
    "cvc": "collocational verb construction",
    "da": "dative",
    "dh": "discourse-level head",
    "dm": "discourse marker",
    "ep": "expletive es",
    "hd": "head",
    "ju": "junctor",
    "mnr": "postnominal modifier",
    "mo": "modifier",
    "ng": "negation",
    "nk": "noun kernel element",
    "nmc": "numerical component",
    "oa": "accusative object",
    "oc": "clausal object",
    "og": "genitive object",
    "op": "prepositional object",
    "par": "parenthetical element",
    "pd": "predicate",
    "pg": "phrasal genitive",
    "ph": "placeholder",
    "pm": "morphological particle",
    "pnc": "proper noun component",
    "rc": "relative clause",
    "re": "repeated element",
    "rs": "reported speech",
    "sb": "subject",
    "sbp": "passivized subject (PP)",
    "sp": "subject or predicate",
    "svp": "separable verb prefix",
    "uc": "unit component",
    "vo": "vocative",
    # Named Entity Recognition
    # OntoNotes 5
    # https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf
    "PERSON": "People, including fictional",
    "NORP": "Nationalities or religious or political groups",
    "FACILITY": "Buildings, airports, highways, bridges, etc.",
    "FAC": "Buildings, airports, highways, bridges, etc.",
    "ORG": "Companies, agencies, institutions, etc.",
    "GPE": "Countries, cities, states",
    "LOC": "Non-GPE locations, mountain ranges, bodies of water",
    "PRODUCT": "Objects, vehicles, foods, etc. (not services)",
    "EVENT": "Named hurricanes, battles, wars, sports events, etc.",
    "WORK_OF_ART": "Titles of books, songs, etc.",
    "LAW": "Named documents made into laws.",
    "LANGUAGE": "Any named language",
    "DATE": "Absolute or relative dates or periods",
    "TIME": "Times smaller than a day",
    "PERCENT": 'Percentage, including "%"',
    "MONEY": "Monetary values, including unit",
    "QUANTITY": "Measurements, as of weight or distance",
    "ORDINAL": '"first", "second", etc.',
    "CARDINAL": "Numerals that do not fall under another type",
    # Named Entity Recognition
    # Wikipedia
    # http://www.sciencedirect.com/science/article/pii/S0004370212000276
    # https://pdfs.semanticscholar.org/5744/578cc243d92287f47448870bb426c66cc941.pdf
    "PER": "Named person or family.",
    "MISC": "Miscellaneous entities, e.g. events, nationalities, products or works of art",
    # https://github.com/ltgoslo/norne
    "EVT": "Festivals, cultural events, sports events, weather phenomena, wars, etc.",
    "PROD": "Product, i.e. artificially produced entities including speeches, radio shows, programming languages, contracts, laws and ideas",
    "DRV": "Words (and phrases?) that are dervied from a name, but not a name in themselves, e.g. 'Oslo-mannen' ('the man from Oslo')",
    "GPE_LOC": "Geo-political entity, with a locative sense, e.g. 'John lives in Spain'",
    "GPE_ORG": "Geo-political entity, with an organisation sense, e.g. 'Spain declined to meet with Belgium'",
}

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
import numpy as np
def pos_counter(row):
  pos_vector = dict.fromkeys(GLOSSARY.keys(), 0)
  doc = nlp(row)
  for d in doc:
    pos_vector[d.pos_] += 1
  return np.asarray(list(pos_vector.values()))

prompt_data['prompt_pos'] = prompt_data['prompt_text'].apply(lambda row: pos_counter(row))
train_data['text_pos'] = train_data['corrected_text'].apply(lambda row: pos_counter(row))

### Entity Recognition

In [ ]:
# Function for matching same entities from prompt and summary
def entity_map(row):
  doc = nlp(row)
  ents = [(e.text, e.label_) for e in doc.ents]
  keys = [e.label_ for e in doc.ents]
  map = {key:[] for key in keys}
  for ent in ents:
    map[ent[1]].append(ent[0])
  return map

prompt_data['prompt_entities'] = prompt_data['prompt_text'].apply(lambda row: entity_map(row))
train_data['summary_entities'] = train_data['corrected_text'].apply(lambda row: entity_map(row))

### Different Words counter

In [ ]:
def different_word_counter(row):
  row = preprocessText(row)
  words_list = []
  for w in row.split(' '):
    if w not in words_list:
      words_list.append(w)

  return len(words_list)


In [ ]:
prompt_data["prompt_text_word_counter"] = prompt_data['prompt_text'].apply(lambda row: different_word_counter(row))
train_data['summary_word_counter'] = train_data['corrected_text'].apply(lambda row: different_word_counter(row))

In [ ]:
# @title Merging the dataset
training_data = train_data.merge(prompt_data, on='prompt_id')

### Different words counter ratio

In [ ]:
def word_counter_ratio(summary_word_counter, prompt_word_counter):
  return summary_word_counter/prompt_word_counter

In [ ]:
training_data['word_counter_ratio'] = training_data.apply(lambda row: word_counter_ratio(row["summary_word_counter"], row["prompt_text_word_counter"]), axis=1)

### Entity Comparison

In [ ]:
def dict_compare(d1, d2):
    d1_keys = set(d1.keys())
    d2_keys = set(d2.keys())
    shared_keys = d1_keys.intersection(d2_keys)
    same = 0
    for key in shared_keys:
      for val1 in d1[key]:
        for val2 in d2[key]:
          if val1 == val2:
            same += 1
    return same

training_data['matching_entities'] = training_data.apply(lambda row: dict_compare(row.prompt_entities, row.summary_entities), axis=1)

### N-grams counting

In [ ]:
def count_cooccurring_ngrams(text, prompt_text):
    cooccurring_count = sum((text & prompt_text).values())
    return cooccurring_count

In [ ]:
for n in range(2, 5):

  text_col = f"{n}grams-text-count"
  corre_col = f"{n}grams-correctedtext-count"
  prompt_col = f"{n}grams-prompttext-count"

  new_col=f"{n}grams-correct-count"
  training_data[new_col] = training_data.apply(lambda row: count_cooccurring_ngrams(row[text_col], row[corre_col]), axis=1)

  new_col=f"{n}grams-cooccurrence-count"
  training_data[new_col] = training_data.apply(lambda row: count_cooccurring_ngrams(row[corre_col], row[prompt_col]), axis=1)

### Semantic Similarity

In [ ]:
def semantic_similarity(text1, text2, preprocess=False):
  if preprocess:
    text1 = preprocessText(text1)
    text2 = preprocessText(text2)

  embedding1 = nlp(text1)
  embedding2 = nlp(text2)

  return embedding1.similarity(embedding2)

In [ ]:
training_data["semantic_similarity"] = training_data.apply(lambda row: semantic_similarity(row["corrected_text_preprocessed"], row["prompt_text_preprocessed"]), axis=1)

<ipython-input-61-34d691352d0d>:9: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  return embedding1.similarity(embedding2)


### Misspelling finding and counting

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()

spell.word_frequency.load_words(token_list)

def misspelled(text):
  words = text.split()
  misspelled = spell.unknown(words)
  return misspelled

In [ ]:
training_data["text_misspelled_words"] = training_data["text_pre_withstop"].apply(lambda x: misspelled(x))
training_data["corrected_misspelled_words"] = training_data["corrected_text_pre_withstop"].apply(lambda x: misspelled(x))

training_data["text_misspelled_counter"] = training_data["text_misspelled_words"].apply(len)
training_data["corrected_misspelled_counter"] = training_data["corrected_misspelled_words"].apply(len)

### Text length ratio

In [ ]:
training_data["length_ratio"] = training_data["text_length"] / training_data["prompt_text_length"]

### TF-IDF score creation

In [ ]:
def add_row(df1, df2, preprocess=False):
  row = df2.unique().tolist()[0]
  if preprocess:
    row = preprocessText(row)
  combined_data = pd.concat([pd.Series([row]),df1.loc[:]]).reset_index(drop=True) #append row on the head of the dataframe
  return combined_data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Group by 'prompt_id' and compute TF-IDF separately for each class
tfidf_vectorizers = {}

for class_id, group in training_data.groupby('prompt_id'):
    #text_data = group['text'].apply(preprocessText)
    text_data = group['corrected_text_preprocessed']

    prompt_question_data = group['prompt_question']
    prompt_title_data = group['prompt_title']
    prompt_text_data = group['prompt_text_preprocessed']

    # Concatenate the preprocessed data for TF-IDF calculation
    combined_data = add_row(text_data, prompt_question_data, True)
    combined_data = add_row(combined_data, prompt_title_data, True)
    combined_data = add_row(combined_data, prompt_text_data)


    # Compute TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data)
    tfidf_vectorizers[class_id] = {'vectorizer': tfidf_vectorizer, 'matrix': tfidf_matrix}

In [ ]:
import sympy

karp_tfidf_scores = {}

# Calculate TF-IDF scores for each document
for class_id, group in training_data.groupby('prompt_id'):
    tfidf_vectorizer = tfidf_vectorizers[class_id]['vectorizer']
    tfidf_matrix = tfidf_vectorizers[class_id]['matrix']
    tfidf_matrix = tfidf_matrix[3:] #remove first 3 rows f the matrix since they belongs to prompt_text, prompt_question, prompt_title

    modulus = sympy.randprime(tfidf_matrix.shape[1]-100, tfidf_matrix.shape[1])

    # Iterate through documents and calculate TF-IDF scores
    for index, row in group.iterrows():
        doc_tfidf = tfidf_matrix[index - group.index[0]].toarray()[0]

        doc_tfidf = doc_tfidf[doc_tfidf>0]

        # Calculate the average TF-IDF score for the document
        #average_tfidf_score = sum(doc_tfidf) / len(doc_tfidf)

        #average_tfidf_scores[index] = average_tfidf_score

        gamma = 1e-2
        single_tfidf_score = sum([t**(gamma*i) for i, t in enumerate(doc_tfidf)]) % modulus

        karp_tfidf_scores[index] = single_tfidf_score

In [ ]:
# Add the calculated average TF-IDF scores as a new column to the DataFrame
training_data['karp_tfidf_scores'] = [karp_tfidf_scores[index] for index in training_data.index]

## Dataset normalization

In [ ]:
# normalize the data taking into consideration the prompt title

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def normalize_col(training_data, col):
  # Create a new DataFrame to store the normalized text length values
  normalized_df = pd.DataFrame()

  # Group by 'prompt_title' and apply the normalization separately for each group
  for title, group in training_data.groupby('prompt_title'):
      normalized_text_length = scaler.fit_transform(group[[col]])
      new_name = "normalized_"+col
      group[new_name] = normalized_text_length
      normalized_df = pd.concat([normalized_df, group])
  training_data = normalized_df.copy()
  return training_data

In [ ]:
normalize_cols = ["text_length", "text_misspelled_counter", "corrected_misspelled_counter", "2grams-cooccurrence-count", "2grams-correct-count",
                  "3grams-cooccurrence-count", "3grams-correct-count", "4grams-cooccurrence-count", "4grams-correct-count", "karp_tfidf_scores", "summary_word_counter"]
for col in normalize_cols:
  training_data = normalize_col(training_data, col)

In [ ]:
normalize_cols.extend(['prompt_text_length'])
del_columns.extend(normalize_cols)

training_data.drop(columns=del_columns, axis=1, inplace=True)

In [ ]:
training_data.rename(columns={"normalized_karp_tfidf_scores": "karp_tfidf_scores"}, inplace=True)

In [ ]:
training_data.head()

,prompt_id,text,content,wording,fixed_summary_text,corrected_text,corrected_text_preprocessed,text_pre_withstop,corrected_text_pre_withstop,text_pos,...,normalized_text_misspelled_counter,normalized_corrected_misspelled_counter,normalized_2grams-cooccurrence-count,normalized_2grams-correct-count,normalized_3grams-cooccurrence-count,normalized_3grams-correct-count,normalized_4grams-cooccurrence-count,normalized_4grams-correct-count,karp_tfidf_scores,normalized_summary_word_counter
3099,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In Egypt, there were many occupations and soci...","In Egypt, there were many occupations and soci...",egypt many occupation social class involved da...,in egypt there were many occupations and socia...,in egypt there were many occupations and socia...,"[15, 24, 10, 20, 0, 12, 22, 2, 56, 2, 10, 22, ...",...,0.076923,0.166667,0.138889,0.349381,0.051151,0.355062,0.020725,0.360335,0.956539,0.477157
3100,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,The highest class was Pharaohs these people we...,The highest class was Pharaohs these people we...,highest class pharaoh people god 2nd highest c...,the highest class was pharaohs these people we...,the highest class was pharaohs these people we...,"[3, 1, 1, 4, 0, 0, 6, 0, 6, 0, 0, 0, 4, 5, 0, ...",...,0.076923,0.083333,0.015152,0.017882,0.012788,0.020804,0.010363,0.023743,0.156903,0.035533
3101,3b9047,The Egyptian society is really different from ...,0.205683,0.380538,The Egyptian society is really different from ...,The Egyptian society is really different from ...,egyptian society really different society lear...,the egyptian society is really different from ...,the egyptian society is really different from ...,"[10, 9, 3, 7, 0, 2, 7, 0, 16, 2, 1, 9, 0, 5, 4...",...,0.076923,0.000000,0.020202,0.089409,0.000000,0.091540,0.000000,0.093575,0.460427,0.111675
3102,3b9047,We have the gods and then Logan and If Logan ...,-1.547163,-1.461245,We have the gods and then Logan and If Logan ...,We have the gods and then Logan and If Logan ...,god logan logan something wh know frank frank ...,we have the gods and then logan and if logan d...,we have the gods and then logan and if logan d...,"[0, 0, 3, 1, 0, 4, 1, 0, 2, 0, 0, 5, 4, 1, 2, ...",...,0.076923,0.083333,0.002525,0.019257,0.000000,0.022191,0.000000,0.025140,0.079536,0.015228
3103,3b9047,The social classes are different because they ...,-0.066112,-0.715083,The social classes are different because they ...,The social classes are different because they ...,social class different different activity like...,the social classes are different because they ...,the social classes are different because they ...,"[5, 13, 0, 5, 0, 3, 9, 1, 19, 1, 4, 4, 2, 7, 3...",...,0.000000,0.000000,0.103535,0.103164,0.089514,0.108183,0.088083,0.113128,0.537596,0.126904


In [ ]:
training_data.corrwith(training_data["content"])

<ipython-input-77-3ecd97e2210c>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  training_data.corrwith(training_data["content"])


content                                    1.000000
wording                                    0.751380
prompt_text_word_counter                  -0.027778
word_counter_ratio                         0.801223
matching_entities                          0.337624
semantic_similarity                        0.537417
length_ratio                               0.777299
normalized_text_length                     0.781834
normalized_text_misspelled_counter         0.275465
normalized_corrected_misspelled_counter    0.194798
normalized_2grams-cooccurrence-count       0.538317
normalized_2grams-correct-count            0.778925
normalized_3grams-cooccurrence-count       0.388556
normalized_3grams-correct-count            0.773245
normalized_4grams-cooccurrence-count       0.338966
normalized_4grams-correct-count            0.768883
karp_tfidf_scores                          0.837049
normalized_summary_word_counter            0.786006
dtype: float64

In [ ]:
training_data.corrwith(training_data["wording"])

<ipython-input-78-360ae61d465e>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  training_data.corrwith(training_data["wording"])


content                                    0.751380
wording                                    1.000000
prompt_text_word_counter                  -0.134943
word_counter_ratio                         0.559691
matching_entities                          0.290207
semantic_similarity                        0.288848
length_ratio                               0.546496
normalized_text_length                     0.556227
normalized_text_misspelled_counter         0.218678
normalized_corrected_misspelled_counter    0.172347
normalized_2grams-cooccurrence-count       0.206662
normalized_2grams-correct-count            0.552565
normalized_3grams-cooccurrence-count       0.051939
normalized_3grams-correct-count            0.546936
normalized_4grams-cooccurrence-count       0.013728
normalized_4grams-correct-count            0.544816
karp_tfidf_scores                          0.566751
normalized_summary_word_counter            0.533349
dtype: float64

In [ ]:
training_data.to_csv('dataset.csv', index=False)